특정 카테고리를 입력 받아 그 카테고리에 대한 팟캐스트를 생성하는 모델을 구현
데이터 수집 -> 보고서 형태로 정리 및 요약 -> 팟캐스트 대본 생성 -> STT 모델 활용 팟캐스트 생성 -> 음성파일 전처리리 -> 팟캐스트 리턴

# ai 모델 MVP 구현

## JSON 데이터 읽기

In [5]:
import json

# JSON 파일 읽기
with open('/home/kkang15634/capstone/AI/data/input/test.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 데이터 구조 확인
print(f"카테고리: {data['category']}")
for idx, article in enumerate(data['articles']):
    print(f"기사 {idx+1}:")
    print(f"제목: {article['title']}")
    print(f"날짜: {article['date']}")
    print(f"내용: {article['text'][:100]}...")
    print("\n")

카테고리: IT/기술
기사 1:
제목: 구글, 차세대 AI 모델 '제미니 3.0' 전격 공개… '인간 수준의 추론 능력' 목표
날짜: 2025-09-11
내용: 구글이 현지 시간 10일, 자사의 연례 개발자 컨퍼런스에서 차세대 대규모 언어 모델(LLM)인 '제미니 3.0'을 공개했습니다. 이번 모델은 텍스트, 이미지, 오디오를 동시에 이해...


기사 2:
제목: 국내 스타트업 'AURA팀', AI 기반 URL 분석 서비스로 CES 2026 혁신상 수상
날짜: 2025-09-11
내용: 대한민국의 AI 스타트업 'AURA팀'이 개발한 동명의 서비스 'AURA'가 세계 최대 IT·가전 전시회 'CES 2026'에서 소프트웨어 및 모바일 앱 부문 혁신상을 수상하는 쾌...


기사 3:
제목: 비동기 처리 기술, 대규모 AI 서비스의 필수 요건으로 자리매김
날짜: 2025-09-10
내용: ChatGPT와 같은 대규모 AI 서비스가 대중화되면서, 수많은 사용자 요청을 효율적으로 처리하기 위한 백엔드 기술의 중요성이 커지고 있습니다. 특히 사용자의 요청을 기다리게 하지...




## Langchain 활용하여 보고서 형태로 제작

In [ ]:
# 보고서 형태로 정리 및 요약

# 팟캐스트 대본 생성 [보고서를 토대로로 팟캐스트 대본 생성]

# STT 모델 활용 팟캐스트 생성 [팟캐스트 대본을 토대로 팟캐스트 생성]

# 음성파일 전처리 [팟캐스트 생성 후 음성파일 전처리]

# 팟캐스트 리턴 [음성파일 전처리 후 팟캐스트 리턴]

<!DOCTYPE html><html lang="ko"><head><title>Google Search</title><style>body{background-color:#fff}</style><script nonce="8MVhKgZTyNY2g-xIFw4PGA">window.google = window.google || {};window.google.c = window.google.c || {ezx:false,cap:0};</script></head><body><noscript><style>table,div,span,p{display:none}</style><meta content="0;url=/httpservice/retry/enablejs?sei=50PCaJuIBaz51e8P5aKzyAw" http-equiv="refresh"><div style="display:block">&#47751; &#52488; &#50504;&#50640; &#51060;&#46041;&#54616;&#51648; &#50506;&#45716; &#44221;&#50864; <a href="/httpservice/retry/enablejs?sei=50PCaJuIBaz51e8P5aKzyAw">&#50668;&#44592;</a>&#47484; &#53364;&#47533;&#54616;&#49464;&#50836;.</div></noscript><script nonce="8MVhKgZTyNY2g-xIFw4PGA">(function(){var sctm=false;(function(){sctm&&google.tick("load","pbsst");}).call(this);})();</script><script nonce="8MVhKgZTyNY2g-xIFw4PGA">//# sourceMappingURL=data:application/json;charset=utf-8;base64,eyJ2ZXJzaW9uIjogMywic291cmNlcyI6WyIiXSwic291cmNlc0NvbnRlbnQiOl